In [ ]:
from cyipopt import minimize_ipopt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import Bounds, LinearConstraint, NonlinearConstraint

In [ ]:
# Fahrzeug
m = 790 #kg
F_MotorMax = 10.41 #kN
alpha = 0.001872 # 1/m

def F_HaftMax(age:str="new"):
    if age=="old":
        return 9.88 #kN
    elif age=="new":
        return 12.27 

# Config Constants
L = 3
B = 1
dL = 1
xStart = 0.1
vStart = 0.2
N=int(L/dL)
num_vars_t =2
optimization_start_t = 1

u = np.zeros((N,num_vars_t)) # 0=x, 1=v
u[0] = [xStart, vStart]


In [ ]:
# Functions
def s(u: np.ndarray, t:int):
    """strecke"""
    x = u[:,0]
    return np.sqrt(dL**2 + (x[t-1]-x[t])**2)

def vm(u:np.ndarray, t:int):
    """durchschnittsgeschwindigkeit"""
    v = u[:,1]
    return (v[t-1] + v[t]) / 2

def dt(u:np.ndarray, t:int):
    """Gebrauchte Zeit für den abschnitt"""
    return s(u, t) / vm(u, t)

def a(u:np.ndarray, t:int):
    """Beschleunigung """
    return (u[t,1]- u[t-1,1]) / dt(u,t)    

In [ ]:
# Objective function
# zeit minimieren
def objective(u:np.ndarray):
    return np.sum([dt(u, t) for t in range(1, L)])

In [ ]:
# Non linear Constraints
def F_Brems_Motor(u:np.ndarray, t:int):
    # u = u.reshape((2,N))
    return m * (a(u, t)+ alpha* vm(u, t)**2)

def F_Reifen(u:np.ndarray, t:int):
    return m* a(u, t)

# To get functions to optimize at each t
# And receive u as passed by ipopt (flat) -> we need to reshape
def F_Brems_Motor_t(t:int):
    return lambda u: F_Brems_Motor(u.reshape((N, num_vars_t)), t)

def F_Reifen_t(t:int):
    return lambda u: F_Reifen(u.reshape((N, num_vars_t)), t)

In [ ]:
def constraints() -> list[NonlinearConstraint]:
    constr = []
    for t in range(optimization_start_t, N):
        constr.append(
            NonlinearConstraint(
            fun=F_Brems_Motor_t(t),
            lb=-np.inf,
            ub=F_MotorMax)
        )
        constr.append(
            NonlinearConstraint(
                fun=F_Reifen_t(t),
                lb=0,
                ub=F_HaftMax
            ) # in den slides ist function und ub **2 aber das ist ja das selbe
        )
        
    return constr

constr = constraints()

In [ ]:
# Bounds are made on input varaibles (x,v) in this case

# 0 <= x(i) <= B
# 0.001 <= v(i) <= infty

bounds = Bounds(
    lb=[0.0 for _ in range(optimization_start_t, N)] + [0.001 for _ in range(optimization_start_t, N)],
    ub=[float(B) for _ in range(optimization_start_t, N)] + [np.inf for _ in range(optimization_start_t, N)]
)
print(bounds)

In [ ]:

result = minimize_ipopt(
    fun=objective, 
    x0=u[optimization_start_t:], 
    bounds=bounds, 
    constraints=constr
)